In [17]:
import pandas as pd
import numpy as np
from os import path
from pathlib import Path

cat = '東証二部'

df = pd.read_csv('./moving_average.csv', encoding='sjis')

file_list = list(Path('./price_data').glob('*csv'))
file_name = path.basename(file_list[-1])
# file_name

l_df = pd.read_csv('./price_data/' + file_name, encoding='sjis')

# 変更
l_df = l_df[['SC', '株価', '市場']]
l_df.drop(l_df.index[0:2], inplace=True)
l_df = l_df[l_df['市場'] == cat]
l_df = l_df[l_df['市場'].isnull() == False]
l_df = l_df[l_df['株価'] != '-']
l_df['株価'] = l_df['株価'].astype(str).astype(np.float64)



# compare_data = pd.merge(df, l_df, on='SC', how='left')
compare_data = pd.merge(df, l_df, on='SC')
compare_data.drop(compare_data.columns[-1], axis=1,inplace=True)

compare_data['compare'] = compare_data[compare_data.columns[-2]] - compare_data[compare_data.columns[-1]]
# compare_data

tmp_data = compare_data.loc[compare_data['compare'] > .0]
tmp_data['rate'] = tmp_data['compare'] / tmp_data[tmp_data.columns[-3]]

tmp_data2 = tmp_data.sort_values(by='rate', ascending=True)[0:30]
# tmp_data2

# 移動平均が上昇傾向のもののみを対象にする
tmp_data2 = tmp_data2[tmp_data2[tmp_data2.columns[2]] < tmp_data2[tmp_data2.columns[3]]]

file_list = list(Path('./financial_data').glob('*csv'))
file_name = path.basename(file_list[-1])
# file_name

financial_data = pd.read_csv('./financial_data/' + file_name, encoding='sjis')
# financial_data

financial_data.drop(financial_data.index[0:2], inplace=True)
financial_data = financial_data[financial_data['PER（予想）'] != '-']
financial_data['PER（予想）'] = financial_data['PER（予想）'].astype(str).astype(np.float64)


C:\Users\ficti\anaconda3\envs\DataScience\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [18]:
financial_data.dtypes

SC             int64
名称            object
市場            object
業種            object
時価総額（百万円）     object
発行済株式数        object
配当利回り         object
1株配当          object
PER（予想）      float64
PBR（実績）       object
EPS（予想）       object
BPS（実績）       object
最低購入額         object
単元株           object
高値日付          object
年初来高値         object
安値日付          object
年初来安値         object
dtype: object

In [19]:
# PBR追加
financial_data = financial_data[financial_data['PBR（実績）'] != '-']
financial_data['PBR（実績）'] = financial_data['PBR（実績）'].astype(str).astype(np.float64)
financial_data = financial_data[financial_data['配当利回り'] != '-']
financial_data['配当利回り'] = financial_data['配当利回り'].astype(str).astype(np.float64)

In [20]:
gyosyu_list = list(financial_data['業種'].unique())

class cat:
    def __init__(self, ave, std):
        self.ave = ave
        self.std = std

dictio = {}
for elm in gyosyu_list:
    # 平均
    #rint(financial_data[financial_data['業種'] == elm]['PER（予想）'].mean()
    #rint(financial_data[financial_data['業種'] == elm]['PER（予想）'].std())
    mean = financial_data[financial_data['業種'] == elm]['PER（予想）'].mean()
    std = financial_data[financial_data['業種'] == elm]['PER（予想）'].std()
    dictio[elm] = cat(mean, std)

financial_data['業種'] = financial_data['業種'].astype(str)

mean_list = []
std_list = []
for elm in list(dictio.keys()):
    mean_list.append(dictio[elm].ave)
    std_list.append(dictio[elm].std)
    

new_dic = {}
new_dic['業種'] = list(dictio.keys())
new_dic['平均'] = mean_list
new_dic['標準偏差'] = std_list
df = pd.DataFrame.from_dict(new_dic)
df['業種'] = df['業種'].astype(str)
# df.dtypes

test_data = pd.merge(financial_data, df, on='業種', how='left')

test_data['PER'] = (test_data['PER（予想）'] - test_data['平均']) / test_data['標準偏差']
# test_data

In [21]:
test_data = test_data[['SC', 'PER', 'PBR（実績）', '配当利回り']]
test_data

,SC,PER,PBR（実績）,配当利回り
0,1301,-1.095410,0.88,2.61
1,1332,-1.038912,0.99,1.74
2,1352,-0.193881,1.19,2.21
3,1376,-0.505918,0.85,1.97
4,1377,0.610172,1.50,0.96
...,...,...,...,...
1736,9983,0.957616,6.40,0.75
1737,9989,-0.419480,2.17,1.96
1738,9993,4.008663,0.67,1.57
1739,9994,-0.578655,0.73,2.00


In [22]:
tmp_data3 = pd.merge(tmp_data2, test_data, on='SC')
tmp_data3

,SC,名称,20200611,20200612,株価,compare,rate,PER,PBR（実績）,配当利回り
0,6870,日本フェンオール,1292.84,1300.40,1300.0,0.40,0.000308,-0.455721,0.65,3.85
1,1434,JESCOホールディングス,330.40,332.12,332.0,0.12,0.000361,-0.154270,0.69,3.61
2,3449,テクノフレックス,1150.68,1164.48,1164.0,0.48,0.000412,-0.368721,1.16,3.52
3,5237,ノザワ,573.96,574.68,574.0,0.68,0.001183,-0.573870,0.41,3.48
4,7587,PALTEK,506.52,508.84,508.0,0.84,0.001651,0.305727,0.59,1.97
5,4623,アサヒペン,1703.68,1703.84,1701.0,2.84,0.001667,-0.516279,0.57,3.23
6,2894,石井食品,216.40,216.60,216.0,0.60,0.002770,1.639424,1.05,1.39
7,8912,エリアクエスト,102.68,103.36,103.0,0.36,0.003483,-0.013728,1.52,3.88
8,3071,ストリーム,73.00,73.40,73.0,0.40,0.005450,1.584150,1.13,0.00
9,4766,ピーエイ,150.40,150.92,150.0,0.92,0.006096,-0.175364,2.16,0.00


In [24]:
save_data = tmp_data3.sort_values(by='PER', ascending=True)[0:7]
save_data

,SC,名称,20200611,20200612,株価,compare,rate,PER,PBR（実績）,配当利回り
10,4115,本州化学工業,1117.08,1119.52,1111.0,8.52,0.007610,-0.616875,0.65,2.88
3,5237,ノザワ,573.96,574.68,574.0,0.68,0.001183,-0.573870,0.41,3.48
11,3435,サンコーテクノ,941.60,944.48,936.0,8.48,0.008978,-0.552099,0.59,2.78
5,4623,アサヒペン,1703.68,1703.84,1701.0,2.84,0.001667,-0.516279,0.57,3.23
0,6870,日本フェンオール,1292.84,1300.40,1300.0,0.40,0.000308,-0.455721,0.65,3.85
2,3449,テクノフレックス,1150.68,1164.48,1164.0,0.48,0.000412,-0.368721,1.16,3.52
9,4766,ピーエイ,150.40,150.92,150.0,0.92,0.006096,-0.175364,2.16,0.00


In [32]:
save_data2 = save_data.copy()
save_data2 = save_data.sort_values(by='PBR（実績）', ascending=True)[0:5]
save_data2

,SC,名称,20200611,20200612,株価,compare,rate,PER,PBR（実績）,配当利回り
3,5237,ノザワ,573.96,574.68,574.0,0.68,0.001183,-0.573870,0.41,3.48
5,4623,アサヒペン,1703.68,1703.84,1701.0,2.84,0.001667,-0.516279,0.57,3.23
11,3435,サンコーテクノ,941.60,944.48,936.0,8.48,0.008978,-0.552099,0.59,2.78
10,4115,本州化学工業,1117.08,1119.52,1111.0,8.52,0.007610,-0.616875,0.65,2.88
0,6870,日本フェンオール,1292.84,1300.40,1300.0,0.40,0.000308,-0.455721,0.65,3.85


In [33]:
save_data3 = save_data2.copy()
save_data3 = save_data2.sort_values(by='配当利回り', ascending=False)[0:3]
save_data3

,SC,名称,20200611,20200612,株価,compare,rate,PER,PBR（実績）,配当利回り
0,6870,日本フェンオール,1292.84,1300.40,1300.0,0.40,0.000308,-0.455721,0.65,3.85
3,5237,ノザワ,573.96,574.68,574.0,0.68,0.001183,-0.573870,0.41,3.48
5,4623,アサヒペン,1703.68,1703.84,1701.0,2.84,0.001667,-0.516279,0.57,3.23


In [36]:
reserve_data = save_data3.iloc[:, 0: -3]

In [37]:
reserve_data

,SC,名称,20200611,20200612,株価,compare,rate
0,6870,日本フェンオール,1292.84,1300.40,1300.0,0.40,0.000308
3,5237,ノザワ,573.96,574.68,574.0,0.68,0.001183
5,4623,アサヒペン,1703.68,1703.84,1701.0,2.84,0.001667
